<a href="https://colab.research.google.com/github/ARnord-Trigger/Electron-Accelerator-Summer2024-25-project-/blob/main/RMbipolarNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd

In [19]:
pf = "/content/RMBipolar02082023_081612.csv"
df = pd.read_csv(pf)

In [20]:
df

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,51,-0.093500,-0.209250,-0.252500,-0.112500,0.111875,0.129875,0.139375,-0.105250,-0.252125,...,0.007500,-0.010000,-0.003250,0.002875,0.003250,0.018125,0.012625,0.001250,0.004125,-0.001500
1,52,-0.111125,-0.224500,-0.277250,-0.126250,0.114750,0.151375,0.166875,-0.114750,-0.290500,...,-0.004500,-0.009500,-0.001625,0.003375,0.003625,0.024250,0.014750,0.004250,0.001500,-0.003875
2,53,-0.125875,-0.262875,-0.211500,-0.099125,0.007625,0.145875,0.211750,-0.085250,-0.274000,...,0.006000,0.002625,-0.000500,0.003125,0.001250,0.019875,0.005125,0.000875,0.004750,0.001000
3,54,-0.113375,-0.245625,-0.204500,-0.084500,-0.005875,0.128000,0.193750,-0.069625,-0.239625,...,-0.003500,-0.006500,0.003125,0.008000,0.005750,0.013750,0.002750,0.002500,-0.003250,-0.000125
4,55,0.073375,0.121625,0.068750,-0.003000,-0.076750,-0.084875,-0.103875,0.068375,0.180500,...,-0.002500,0.011750,0.003250,-0.003375,-0.002250,-0.010750,-0.007625,0.000000,-0.000500,0.001500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,134,-0.002125,-0.003875,-0.008375,-0.000875,0.004250,0.000875,-0.001875,-0.003375,0.000375,...,0.016125,0.158125,0.113000,0.177000,0.131250,0.099125,0.042375,-0.028750,-0.218750,-0.103375
84,135,-0.002625,-0.011500,-0.007125,-0.000250,0.007875,-0.000750,-0.001750,-0.002375,0.001375,...,-0.015750,0.101625,0.096875,0.186125,0.113250,0.127375,0.090875,-0.002000,-0.178875,-0.105250
85,136,0.001875,0.000250,0.002625,0.001875,-0.001000,-0.003250,-0.003750,0.003625,0.005375,...,-0.108500,-0.160250,-0.026125,0.083500,0.110625,0.136125,0.193875,0.062750,0.090250,-0.023750
86,137,-0.002125,-0.007625,-0.009500,-0.001250,0.005000,0.002125,-0.000750,-0.000750,-0.003125,...,-0.125000,-0.218625,-0.065375,0.043250,0.107500,0.189000,0.208875,0.079625,0.168750,0.003750


In [21]:
RM = df.values
RM=RM[:,1:]

In [22]:
I_1 = np.identity(48)
I_2 = np.identity(40)
BPI_x1 = RM[0:49,0:57].T
BPI_x2 = RM[49:89,0:57].T
BPI_y1 = RM[0:49,57:113].T
BPI_y2 = RM[49:89,57:113].T

In [71]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layer_dims):
        self.layer_dims = layer_dims
        self.parameters = self.initialize_parameters(layer_dims)
        self.cache = {}
        self.grads = {}
        self.epsilon = 1e-8  # Small number to avoid division by zero in batch normalization
        self.beta = 0.9  # Parameter for the Adam optimizer
        self.beta1 = 0.9  # Parameter for Adam optimizer
        self.beta2 = 0.999  # Parameter for Adam optimizer
        self.learning_rate = 0.001  # Learning rate for Adam optimizer

    def initialize_parameters(self, layer_dims):
        parameters = {}
        L = len(layer_dims)

        for l in range(1, L):
            parameters[f"W{l}"] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2 / layer_dims[l-1])  # He initialization
            parameters[f"b{l}"] = np.zeros((layer_dims[l], 1))

        return parameters

    def relu(self, Z):
        return np.maximum(0, Z)

    def softmax(self, Z):
        exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

    def forward_propagation(self, X):
        A = X
        L = len(self.layer_dims) - 1
        self.cache["A0"] = X
        for l in range(1, L):
            W = self.parameters[f"W{l}"]
            b = self.parameters[f"b{l}"]
            Z = np.dot(W, A) + b
            A = self.relu(Z)
            self.cache[f"Z{l}"] = Z
            self.cache[f"A{l}"] = A

            # Batch normalization
            mean = np.mean(A, axis=1, keepdims=True)
            var = np.var(A, axis=1, keepdims=True)
            A_normalized = (A - mean) / np.sqrt(var + self.epsilon)
            self.cache[f"A{l}_normalized"] = A_normalized

        # Output layer
        WL = self.parameters[f"W{L}"]
        bL = self.parameters[f"b{L}"]
        ZL = np.dot(WL, A) + bL
        AL = self.softmax(ZL)
        self.cache[f"Z{L}"] = ZL
        self.cache[f"A{L}"] = AL
        # print(AL.shape)

        return AL

    def compute_loss(self, AL, Y):
        m = Y.shape[1]
        cost = np.sum((AL-Y)**2) / m  # Cross-entropy loss
        return cost

    def backward_propagation(self, X, Y):
        m = Y.shape[1]
        L = len(self.layer_dims) - 1
        AL = self.cache[f"A{L}"]
        dAL = AL - Y

        # Output layer gradients
        dZL = dAL
        self.grads[f"dW{L}"] = np.dot(dZL, self.cache[f"A{L-1}"].T) / m
        self.grads[f"db{L}"] = np.sum(dZL, axis=1, keepdims=True) / m

        dA_prev = np.dot(self.parameters[f"W{L}"].T, dZL)

        # Hidden layers gradients
        for l in reversed(range(1, L)):
            dZ = dA_prev * (self.cache[f"Z{l}"] > 0).astype(int)  # ReLU derivative
            self.grads[f"dW{l}"] = np.dot(dZ, self.cache[f"A{l-1}"].T) / m
            self.grads[f"db{l}"] = np.sum(dZ, axis=1, keepdims=True) / m

            # Batch normalization gradients
            dA_prev = np.dot(self.parameters[f"W{l}"].T, dZ)

    def update_parameters(self):
        L = len(self.layer_dims) - 1

        for l in range(1, L + 1):
            self.parameters[f"W{l}"] -= self.learning_rate * self.grads[f"dW{l}"]
            self.parameters[f"b{l}"] -= self.learning_rate * self.grads[f"db{l}"]

    def train(self, X, Y, epochs=100, batch_size=12):
        m = X.shape[1]

        for epoch in range(epochs):
            for i in range(0, m, batch_size):
                X_batch = X[:, i:i+batch_size]
                Y_batch = Y[:, i:i+batch_size]

                # Forward propagation
                AL = self.forward_propagation(X_batch)

                # Compute cost
                # print(f"AL:{AL.shape}")
                # print(f"Y_batch:{Y_batch.shape}")
                cost = self.compute_loss(AL, Y_batch)

                # Backward propagation
                self.backward_propagation(X_batch, Y_batch)

                # Update parameters
                self.update_parameters()

                if i % 1000 == 0:
                    print(f"Epoch {epoch+1}/{epochs}, Batch {i//batch_size+1}/{m//batch_size}, Cost: {cost:.4f}")


In [69]:
I_1 = np.identity(48)
I_2 = np.identity(40)
BPI_x1 = RM[0:49,0:57].T
BPI_x2 = RM[49:89,0:57].T
BPI_y1 = RM[0:49,57:113].T
BPI_y2 = RM[49:89,57:113].T

In [43]:
X_train = I_1 = np.identity(48)
Y_train = BPI_x1 = RM[0:48,0:56].T
layers_dims = [48,108,108,96,96,64,56]
print(X_train.shape)
print(Y_train.shape)

(48, 48)
(56, 48)


In [74]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# y_train = y_train.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)


# Normalize the input data (optional)
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
# X_test = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)

# Define the architecture of the neural network
layer_dims = [X_train.shape[0], 108, 108, 96,56]  # Input dimension, hidden layer dimensions, output dimension
epochs = 190

batch_size = 12

# Initialize and train the neural network
model = NeuralNetwork(layer_dims)
model.train(X_train, Y_train, epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test set
def accuracy(predictions, labels):
    return np.mean(np.argmax(predictions, axis=0) == np.argmax(labels, axis=0))

# Forward pass on the test set
# predictions = model.forward_propagation(X_test)
# acc = accuracy(predictions, y_test_encoded)

# print(f"Test Accuracy: {acc * 100:.2f}%")


Epoch 1/190, Batch 1/4, Cost: 1.1840
Epoch 2/190, Batch 1/4, Cost: 1.1812
Epoch 3/190, Batch 1/4, Cost: 1.1786
Epoch 4/190, Batch 1/4, Cost: 1.1761
Epoch 5/190, Batch 1/4, Cost: 1.1737
Epoch 6/190, Batch 1/4, Cost: 1.1714
Epoch 7/190, Batch 1/4, Cost: 1.1691
Epoch 8/190, Batch 1/4, Cost: 1.1669
Epoch 9/190, Batch 1/4, Cost: 1.1647
Epoch 10/190, Batch 1/4, Cost: 1.1626
Epoch 11/190, Batch 1/4, Cost: 1.1606
Epoch 12/190, Batch 1/4, Cost: 1.1587
Epoch 13/190, Batch 1/4, Cost: 1.1568
Epoch 14/190, Batch 1/4, Cost: 1.1550
Epoch 15/190, Batch 1/4, Cost: 1.1533
Epoch 16/190, Batch 1/4, Cost: 1.1515
Epoch 17/190, Batch 1/4, Cost: 1.1497
Epoch 18/190, Batch 1/4, Cost: 1.1479
Epoch 19/190, Batch 1/4, Cost: 1.1462
Epoch 20/190, Batch 1/4, Cost: 1.1445
Epoch 21/190, Batch 1/4, Cost: 1.1428
Epoch 22/190, Batch 1/4, Cost: 1.1413
Epoch 23/190, Batch 1/4, Cost: 1.1397
Epoch 24/190, Batch 1/4, Cost: 1.1382
Epoch 25/190, Batch 1/4, Cost: 1.1367
Epoch 26/190, Batch 1/4, Cost: 1.1353
Epoch 27/190, Batch 1